# Volumetric Surface Reconstruction.

### Data Loader (Dino_images.py)

### Calculating the Fundamental Matrix F

Given two non-calibrated cameras, define camera center as $C, C'$ respectively, homogenous image points $x, x'$ respectively, and corresponding homogenous 3D point as $X$.

The goal is to find a transformation F that maps an image point in the first camera to an epipolar line in the second camera. In other words,

$$l' = Fx$$

To obtain the epipole of the right image, project camera center $C$ in the first camera to the second camera's image plane,

$$e' = P'C$$

...

Finally, the fundamental matrix is defined as:

$$F = [e']_{\times}P'P^+$$


### Finding Matching Pairs (openCV)

### Pair Matching

For a given dots on the first images denoted by $x_i$, is possible to find a corrospounding epipolar line $l_i$ on the second image given the fundimental matrix $F$. The navie approach is to perform pattern matching using colour intensity along the epipolar line. However, it is still difficult challenging to find the true matching pairs on the second images due to the following reasons:
* Error in fundimental matrix estimation
* Error in epipolar line $l_i$ estimation

Hence, it is very important to come up with a robust way of finding the matched point.

#### Convolutional Absolute Intensity Difference Method

First of all, we try ... **MIKU FILL THIS PART**

#### Normalized Linear Kernal Correlation Coefficient Method

Secondly, we tries to implement the correlation coefficient method described in the [https://docs.opencv.org/4.x/df/dfb/group__imgproc__object.html#gga3a7850640f1fe1f58fe91a2d7583695daf9c3ab9296f597ea71f056399a5831da](CCOFE_NORM).  Assume that $T(x, y), I(x, y)$ gives the pixel intensity on the template / matching image with position $(x, y)$. Similarity score $R(x, y)$ is given by the normalization function $R(x, y) = \frac{\sum_{x', y'}(T(x', y') \cdot I'(x+x', y+y'))}{\sqrt{\sum_{x', y'}T(x', y')^2 \cdot I'(x+x', y+y')^2}}$ by convolutionaly going through the image. Here $T'(x', y') = M(x', y') \cdot (T(x', y') - \frac{\sum_{x'', y''}(T(x'', y'') \cdot M(x'', y''))}{\sum_{x'', y''}T(x'', y'')})$, $I(x' + x, y' + y) = M(x', y') \cdot (I(x' + x, y' + y) - \frac{\sum_{x'', y''}(I(x'' + x, y'' + y) \cdot M(x'', y''))}{\sum_{x'', y''}T(x'', y'')})$

We first calculate the score matrix, then the goal is to optimize the score along and near the epipolar line $y = l(x)$. We implemented a pair wise optimization function using the correlation coefficient score matrix $\mathbb{S}$ and horizontal shift $f$ from the epipolar line. We are trying to optimize the following by finding point $x, y$:
$$
L = \mathbb{S}_{x + f, l(x) + f} \cdot (1 - |f| \cdot epsilon)
$$

### Triangulation

Given a pair of homogenous matching points $x,x'$ that corresponds to an unknown homogenous 3D point $\mathbf{X}$ and projection matrices $P,P'$, triangulation can be done in order to determine the 3D point $\mathbf{X}$.

Define

$$
\begin{equation*}
P = 
\begin{bmatrix}
r_{11} & r_{12} & r_{13} & t_{1} \\
r_{21} & r_{22} & r_{23} & t_{2} \\
r_{31} & r_{32} & r_{33} & t_{3}
\end{bmatrix}
\end{equation*}
$$

$$
\begin{equation*}
P' = 
\begin{bmatrix}
r_{11}' & r_{12}' & r_{13}' & t_{1}' \\
r_{21}' & r_{22}' & r_{23}' & t_{2}' \\
r_{31}' & r_{32}' & r_{33}' & t_{3}'
\end{bmatrix}
\end{equation*}
$$

and 

$$
\mathbf{X} =
\begin{bmatrix}
X \\
Y \\
Z \\
1 
\end{bmatrix}$$

and

$$
x =
\begin{bmatrix}
u \\
v \\
1 
\end{bmatrix}$$

$$
x' =
\begin{bmatrix}
u' \\
v' \\
1 
\end{bmatrix}$$

Then by definition,

$$x = PX$$ $$x' = P'X$$

Thus, taking the cross product results in:

$$x \times PX = 0$$ $$x' \times PX' = 0$$

Rewriting in terms of the projection matrices' rows yields:

$$
\begin{array}{cc}
x(\mathbf{p_3}^T \mathbf{X})-(\mathbf{p_1}^T \mathbf{X}) \\
y(\mathbf{p_3}^T \mathbf{X})-(\mathbf{p_2}^T \mathbf{X}) \\
x(\mathbf{p_2}^T \mathbf{X})-y(\mathbf{p_1}^T \mathbf{X})
\end{array}
$$

Taking the two linearly independent rows and using the two image points yields the following system of linear equations:

$$
A = \left[ \begin{array}{cc}
x(\mathbf{p_3}^T)-(\mathbf{p_1}^T) \\
y(\mathbf{p_3}^T)-(\mathbf{p_2}^T) \\
x'(\mathbf{p_3}^T)-(\mathbf{p_1}^T) \\
y'(\mathbf{p_3}^T)-(\mathbf{p_2}^T)
\end{array}
\right]
$$

where we are solving for $A\mathbf{X} = 0$.

Using the homogenous route, the vector corresponding to the smallest singular value of $A$ is the solution for $\mathbf{X}$

### Sequential Update

### References

- Ghosh, Sourish. “Calculating the Fundamental Matrix from Camera Matrices.” Sourish Ghosh, https://sourishghosh.com/2016/fundamental-matrix-from-camera-matrices/.
- Wu, Kai. Triangulation, https://imkaywu.github.io/blog/2017/07/triangulation/. 